In [1]:
import pandas as pd
import transformers

In [2]:
import torch

device=torch.device("cpu")
#device=torch.device("cuda")

There are 1 GPU(s) available.
We will use the GPU: NVIDIA GeForce GT 710


/home/user/.local/lib/python3.6/site-packages/torch/cuda/__init__.py:83: UserWarning: 
    Found GPU%d %s which is of cuda capability %d.%d.
    PyTorch no longer supports this GPU because it is too old.
    The minimum cuda capability supported by this library is %d.%d.
    
  warnings.warn(old_gpu_warn.format(d, name, major, minor, min_arch // 10, min_arch % 10))


In [8]:
from transformers import BertTokenizer
from transformers import BertForSequenceClassification
tokenizer = BertTokenizer.from_pretrained('Repository/biobert/tokenizer_biobert', do_lower_case=True)
model = BertForSequenceClassification.from_pretrained(
     "Repository/biobert/model_biobert-finetuning",
     num_labels = 2, 
     output_attentions = False, # Whether the model returns attentions weights.
     output_hidden_states = True, # Whether the model returns all hidden-states.
).to(device)

In [9]:
import tqdm
model.eval()
with torch.no_grad():
 f = open("data/masked_sentences.txt","r")
 f2 = open("data/masked_sentences_bert.txt","w")
 m = torch.nn.Softmax()
 for line in tqdm.tqdm(f):
   ls = line.strip().split("\t")
   sent = ls[5]
   if len(sent.split(" ")) > 500:continue
   indexed_tokens = tokenizer.encode(
                        sent,                     # Sentence to encode.
                        add_special_tokens = True)
   tokens_tensor = torch.tensor([indexed_tokens])
   if len(tokens_tensor[0]) > 500:continue
   all_encoder_layers = model(tokens_tensor.to(device))
   out= m(all_encoder_layers[0])
   judge = int(out[0][1] > 0.5)
   if judge ==1:
     f2.write(str(out[0][1].item())+"\t"+line)
 f2.close() 

/home/user/.local/lib/python3.6/site-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


0it [00:00, ?it/s]

/home/user/.local/lib/python3.6/site-packages/ipykernel_launcher.py:17: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
